In [1]:
from spleeter.separator import Separator
separator = Separator('spleeter:2stems')
audio_file = 'Can you solve the jail break riddle_ - Dan Finkel.mp4'
separator.separate_to_file(audio_file, 'split')

INFO:tensorflow:Using config: {'_model_dir': 'pretrained_models\\2stems', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.7
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead
INFO:tensorflow:Calling model_fn.
INFO:tensorfl

In [3]:
# !conda install pytorch==2.0.0 torchaudio==2.0.0 pytorch-cuda=11.8 -c pytorch -c nvidia

Solving environment: ...working... done

# All requested packages already installed.



In [1]:
import whisperx
device = "cuda"

batch_size = 4 
compute_type = "float16"
audio = whisperx.load_audio("split/Can you solve the jail break riddle_ - Dan Finkel/vocals.wav")
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

result = model.transcribe(audio, batch_size=batch_size)

# iresult  = result["segments"]

model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)
diarize_model = whisperx.DiarizationPipeline(use_auth_token="",device=device)
diarize_segments = diarize_model(audio)

result = whisperx.assign_word_speakers(diarize_segments, result)
print(diarize_segments)
print(result["segments"]) 


c:\Users\tejas\anaconda3\envs\vd\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
torchvision is not available - cannot save figures


No language specified, language will be first be detected for each audio file (increases inference time).


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.3.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\tejas\.cache\torch\whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.0. Bad things might happen unless you revert torch to 1.x.
Detected language: en (1.00) in first 30s of audio...
                              segment label     speaker       start  \
0   [ 00:00:06.697 -->  00:00:26.918]     A  SPEAKER_01    6.697793   
1   [ 00:00:27.563 -->  00:00:33.896]     B  SPEAKER_01   27.563667   
2   [ 00:00:34.286 -->  00:00:46.833]     C  SPEAKER_01   34.286927   
3   [ 00:00:47.784 -->  00:01:14.677]     D  SPEAKER_01   47.784380   
4   [ 00:01:15.135 -->  00:01:22.996]     E  SPEAKER_01   75.135823   
5   [ 00:01:22.996 -->  00:01:24.524]     F  SPEAKER_00   82.996604   
6   [ 00:01:35.186 -->  00:01:35.220]     G  SPEAKER_01   95.186757   
7   [ 00:01:35.220 -->  00:01:37.003]     H  SPEAKER_00   95.220713   
8   [ 00:01:37.003 -->  00:01:42.809]     I  SPEAKER_01   97.00339

In [ ]:
speakers = diarize_segments.speaker.unique()

In [19]:
result['segments']

[{'start': 6.755,
  'end': 11.277,
  'text': ' Your timing made you and your partner the most infamous bank robbers in the West.',
  'words': [{'word': 'Your',
    'start': 6.755,
    'end': 6.955,
    'score': 0.694,
    'speaker': 'SPEAKER_01'},
   {'word': 'timing',
    'start': 7.055,
    'end': 7.535,
    'score': 0.958,
    'speaker': 'SPEAKER_01'},
   {'word': 'made',
    'start': 7.615,
    'end': 7.855,
    'score': 0.767,
    'speaker': 'SPEAKER_01'},
   {'word': 'you',
    'start': 7.915,
    'end': 8.075,
    'score': 0.896,
    'speaker': 'SPEAKER_01'},
   {'word': 'and',
    'start': 8.116,
    'end': 8.176,
    'score': 0.999,
    'speaker': 'SPEAKER_01'},
   {'word': 'your',
    'start': 8.216,
    'end': 8.336,
    'score': 0.788,
    'speaker': 'SPEAKER_01'},
   {'word': 'partner',
    'start': 8.376,
    'end': 8.756,
    'score': 0.827,
    'speaker': 'SPEAKER_01'},
   {'word': 'the',
    'start': 8.816,
    'end': 8.896,
    'score': 0.841,
    'speaker': 'SPEAKER_

In [21]:
result['segments'][0]['speaker']

'SPEAKER_01'

In [31]:
segments_list = []
accumulated_content = ""

# Define a maximum duration threshold (e.g., 10 seconds) for splitting segments
max_duration = 10.0

for i in range(len(result['segments'])):
    current_segment = result['segments'][i]
    
    # Get the previous segment text (if it exists)
    previous_segment_text = result['segments'][i-1]['text'] if i > 0 else ''
    
    # Add the current segment text to the accumulated content
    accumulated_content += " " + current_segment['text']
    
    # Get the next segment text (if it exists)
    next_segment_text = result['segments'][i+1]['text'] if i < len(result['segments']) - 1 else ''
    
    # Calculate the duration of the segment
    duration = current_segment['end'] - current_segment['start']
    
    # Get speaker information
    speaker = current_segment.get('speaker', 'Unknown')
    
    # Split the segment if the duration exceeds the maximum allowed
    if duration > max_duration:
        # Determine the midpoint to split the segment
        midpoint = len(current_segment['text']) // 2
        part1_text = current_segment['text'][:midpoint].strip()
        part2_text = current_segment['text'][midpoint:].strip()
        
        # First part of the split segment
        segment_info_part1 = {
            'text': part1_text,
            'start_time': current_segment['start'],
            'end_time': current_segment['start'] + duration / 2,
            'duration': duration / 2,
            'accumulated_content': accumulated_content.strip(),
            'next_text': part2_text,
            'speaker': speaker,
        }
        segments_list.append(segment_info_part1)
        
        # Second part of the split segment
        segment_info_part2 = {
            'text': part2_text,
            'start_time': current_segment['start'] + duration / 2,
            'end_time': current_segment['end'],
            'duration': duration / 2,
            'accumulated_content': accumulated_content.strip(),
            'next_text': next_segment_text,
            'speaker': speaker,
        }
        segments_list.append(segment_info_part2)
    else:
        # Store each segment's text, duration, accumulated content, and context in a dictionary
        segment_info = {
            'text': current_segment['text'],
            'start_time': current_segment['start'],
            'end_time': current_segment['end'],
            'duration': duration,
            'accumulated_content': accumulated_content.strip(),
            'next_text': next_segment_text,
            'speaker': speaker,
        }
        segments_list.append(segment_info)

# Example of How to Use the Data in a Prompt
for segment in segments_list:
    prompt = f"""
    Translate the following English segment into Hindi while preserving the context, maintaining the narrative flow, trying to stay within the duration, and keeping proper names in their original form:
    {segment['text']}

    Context:
    Accumulated Content: {segment['accumulated_content']}
    Next Segment: {segment['next_text']}
    Speaker: {segment['speaker']}

    Segment Information:
    Start Time: {segment['start_time']} seconds
    End Time: {segment['end_time']} seconds
    Duration: {segment['duration']} seconds
    """
    print(prompt)



    Translate the following English segment into Hindi while preserving the context, maintaining the narrative flow, trying to stay within the duration, and keeping proper names in their original form:
     Your timing made you and your partner the most infamous bank robbers in the West.

    Context:
    Accumulated Content: Your timing made you and your partner the most infamous bank robbers in the West.
    Next Segment: Now, you'll need to use that timing to help you break out of jail.
    Speaker: SPEAKER_01

    Segment Information:
    Start Time: 6.755 seconds
    End Time: 11.277 seconds
    Duration: 4.521999999999999 seconds
    

    Translate the following English segment into Hindi while preserving the context, maintaining the narrative flow, trying to stay within the duration, and keeping proper names in their original form:
    Now, you'll need to use that timing to help you break out of jail.

    Context:
    Accumulated Content: Your timing made you and your partner

In [40]:
import openai

# Initialize the OpenAI API key
openai.api_key = ""

# Assuming segments_list is already defined with accumulated_content and other details
translations = []

for i, segment in enumerate(segments_list):
    # Get the previous segment text if it exists
    previous_segment_text = segments_list[i-1]['text'] if i > 0 else ''
    
    # Prepare the prompt with the current segment and context
    prompt = f"""
    Translate the following English segment into Hindi while preserving the context, maintaining the narrative flow, staying within the given duration, and keeping proper names in their original form:

    {segment['text']}
    """
    
    # Send the prompt to GPT-3.5-turbo for translation using the client
    chat_completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
    )
    
    # Extract and store the translation with start and end time
    translated_text = chat_completion.choices[0].message['content'].strip()
    
    # Store only the translation, time details, and speaker
    translation = {
        'text': translated_text,
        'start_time': segment['start_time'],
        'end_time': segment['end_time'],
        'speaker': segment['speaker']  # Include the speaker information
    }
    translations.append(translation)

# Output the translations with start time, end time, and speaker
for i, translation in enumerate(translations):
    print(f"Segment {i+1}: {translation['text']}")
    print(f"Speaker: {translation['speaker']}")
    print(f"Start Time: {translation['start_time']} seconds, End Time: {translation['end_time']} seconds\n")

# If you only want the translations:
# for i, translation in enumerate(translations):
#     print(translation['text'])


Segment 1: तुम्हारा समयिंग तुम्हें और तुम्हारे साथी को पश्चिम में सबसे बदनाम बैंक डाकू बना दिया।
Speaker: SPEAKER_01
Start Time: 6.755 seconds, End Time: 11.277 seconds

Segment 2: अब, आपको उस समय का उपयोग करना होगा जिसे आपको जेल से निकलने में मदद मिलेगी।
Speaker: SPEAKER_01
Start Time: 11.677 seconds, End Time: 15.218 seconds

Segment 3: निर्धारित समय पर, आप इलेक्ट्रिक फेंस के पास यार्ड में चल रहे होंगे।
Speaker: SPEAKER_01
Start Time: 15.818 seconds, End Time: 19.94 seconds

Segment 4: आपका साथी आपको संकेत देगा, और बिल्कुल 45 सेकंड बाद, बाड़ा सर्किट बाहर कर देगा।
Speaker: SPEAKER_01
Start Time: 20.54 seconds, End Time: 26.743 seconds

Segment 5: यह एक सेकंड या दो के बाद स्वचालित रूप से पुनः शुरू हो जाएगा, लेकिन जब तक आप जल्दी चलते हैं, तब तक आप घर में सुरक्षित हैं।
Speaker: SPEAKER_01
Start Time: 27.683 seconds, End Time: 33.765 seconds

Segment 6: और फिर आपको आपकी भयानकता से ये पता चलता है कि आपकी घड़ी टूट गई है, और इसे ठीक करने का कोई समय नहीं है।
Speaker: SPEAKER_01
Start Time: 34

In [41]:
for i, translation in enumerate(translations):
    print(translation['text'])

तुम्हारा समयिंग तुम्हें और तुम्हारे साथी को पश्चिम में सबसे बदनाम बैंक डाकू बना दिया।
अब, आपको उस समय का उपयोग करना होगा जिसे आपको जेल से निकलने में मदद मिलेगी।
निर्धारित समय पर, आप इलेक्ट्रिक फेंस के पास यार्ड में चल रहे होंगे।
आपका साथी आपको संकेत देगा, और बिल्कुल 45 सेकंड बाद, बाड़ा सर्किट बाहर कर देगा।
यह एक सेकंड या दो के बाद स्वचालित रूप से पुनः शुरू हो जाएगा, लेकिन जब तक आप जल्दी चलते हैं, तब तक आप घर में सुरक्षित हैं।
और फिर आपको आपकी भयानकता से ये पता चलता है कि आपकी घड़ी टूट गई है, और इसे ठीक करने का कोई समय नहीं है।
सिग्नल आ रहा है, और अगर आप 45 सेकंड की गिनती में भी एक छोटी सी गलती करते हैं, तो आप पक जाएंगे।
अपनी जेबों में खोजते हुए, आप कुछ ऐसा खोजते हैं जो मदद कर सकता है— एक लाइटर और दो फ्यूज जो आपने जेल कार्यक्रम में पहले बनाए थे।
प्रत्येक फ्यूज एक ज्वलनशील दोरी का ढेर है जिसे एक ही धागे पर जलाने के लिए बनाया गया है और सटीक एक मिनट के लिए जलाने के लिए।
समस्या यह है कि वैसे भी एक्सरी यूनिफॉर्म दिखते हैं, लेकिन वे बराबर जलते नहीं।
तो अगर आप एक को आधे में काट दें, उदाहरण के 

In [5]:

# !git clone https://github.com/serp-ai/bark-with-voice-clone
# %cd bark-with-voice-clone/
# !pip install git+https://github.com/suno-ai/bark.git

In [38]:
from tqdm import tqdm
import torch
from pydub import AudioSegment
import os

import scipy
from bark import SAMPLE_RATE, generate_audio, preload_models
from bark.api import generate_audio
from transformers import BertTokenizer
from bark.generation import SAMPLE_RATE, preload_models, codec_decode, generate_coarse, generate_fine, generate_text_semantic
preload_models()

In [42]:
import os
import scipy.io.wavfile

# Sample rate for saving the audio
SAMPLE_RATE = SAMPLE_RATE

# Get the unique speakers from diarize_segments
unique_speakers = diarize_segments.speaker.unique()

# Dynamically create a speaker mapping based on the unique speakers
speaker_mapping = {}
for i, speaker in enumerate(unique_speakers):
    speaker_mapping[speaker] = f"v2/hi_speaker_{i}"

# Directory to save audio files
output_dir = "translated_audio_segments"
os.makedirs(output_dir, exist_ok=True)

# Generate and save audio for each translated segment
for translation in translations:
    start_time_str = f"{translation['start_time']:.2f}"
    end_time_str = f"{translation['end_time']:.2f}"
    filename = f"{start_time_str}-{end_time_str}.wav"
    output_path = os.path.join(output_dir, filename)
    
    # Assign the correct TTS voice based on the speaker in the translation
    speaker_label = translation.get('speaker', None)
    history_prompt = speaker_mapping.get(speaker_label, "v2/hi_speaker_1")  # Default to Speaker 0 if not found
    
    # Generate audio based on the translation text and the selected speaker
    tr_audio = generate_audio(translation['text'], history_prompt=history_prompt)
    
    # Save the generated audio
    scipy.io.wavfile.write(output_path, rate=SAMPLE_RATE, data=tr_audio)
    
    print(f"Saved audio: {filename}")


100%|██████████| 19/19 [01:08<00:00,  3.61s/it]


Saved audio: 6.75-11.28.wav


100%|██████████| 23/23 [01:15<00:00,  3.29s/it]


Saved audio: 11.68-15.22.wav


100%|██████████| 15/15 [00:51<00:00,  3.43s/it]


...

Saved audio: 163.09-167.20.wav


100%|██████████| 27/27 [00:30<00:00,  1.12s/it]


Saved audio: 167.84-174.66.wav


100%|██████████| 17/17 [00:25<00:00,  1.48s/it]


Saved audio: 175.54-179.71.wav


In [43]:
from moviepy.editor import VideoFileClip, AudioFileClip

video = VideoFileClip("Can you solve the jail break riddle_ - Dan Finkel.mp4")
video_no_audio = video.without_audio()
background_audio = AudioFileClip("split/Can you solve the jail break riddle_ - Dan Finkel/accompaniment.wav")
video_with_new_audio = video_no_audio.set_audio(background_audio)
video_with_new_audio.write_videofile("output_video_with_new_audio.mp4")


Moviepy - Building video output_video_with_new_audio.mp4.
MoviePy - Writing audio in output_video_with_new_audioTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video_with_new_audio.mp4



Moviepy - Done !
Moviepy - video ready output_video_with_new_audio.mp4


In [44]:
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips, vfx
import os

# Directory containing the audio segments
audio_dir = 'translated_audio_segments'

# Load the silent video
video = VideoFileClip('output_video_with_new_audio.mp4')

# Get the list of all audio files
audio_files = [f for f in os.listdir(audio_dir) if f.endswith('.wav')]

# Sort files based on their start time, accounting for missing segments
audio_files.sort(key=lambda x: float(x.split('-')[0]))

# List to hold the modified video segments
video_segments = []

# Initialize the previous end time to handle gaps
previous_end_time = 0.0

for audio_file in audio_files:
    # Extract start and end times from the audio file name
    start_time, end_time = map(float, audio_file[:-4].split('-'))
    
    # Handle missing segments by checking for gaps between the current start time and the previous end time
    if start_time > previous_end_time:
        # Add a silent video segment for the missing time
        missing_segment = video.subclip(previous_end_time, start_time)
        video_segments.append(missing_segment)
    
    # Calculate the target duration of the audio file
    audio = AudioFileClip(os.path.join(audio_dir, audio_file))
    target_duration = audio.duration
    
    # Extract the corresponding video segment
    video_segment = video.subclip(start_time, end_time)
    
    # Calculate the current duration of the video segment
    current_duration = video_segment.duration
    
    # Adjust the speed of the video segment
    speed_factor = current_duration / target_duration
    modified_segment = video_segment.fx(vfx.speedx, speed_factor)
    
    # Set the audio to the modified video segment
    modified_segment = modified_segment.set_audio(audio)
    
    # Add the modified segment to the list
    video_segments.append(modified_segment)
    
    # Update the previous end time
    previous_end_time = end_time

# Handle the final gap if the last segment ends before the video ends
if previous_end_time < video.duration:
    final_segment = video.subclip(previous_end_time, video.duration)
    video_segments.append(final_segment)

# Concatenate all the modified video segments
final_video = concatenate_videoclips(video_segments)

# Save the final video
final_video.write_videofile('video_with_audio.mp4', codec="libx264")


Moviepy - Building video video_with_audio.mp4.
MoviePy - Writing audio in video_with_audioTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video_with_audio.mp4



Moviepy - Done !
Moviepy - video ready video_with_audio.mp4


In [45]:

final_video = final_video.fx(vfx.speedx, 1)
final_video.write_videofile('output_video_with_audio_1.5x.mp4', codec="libx264")


Moviepy - Building video output_video_with_audio_1.5x.mp4.
MoviePy - Writing audio in output_video_with_audio_1.5xTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video_with_audio_1.5x.mp4



Moviepy - Done !
Moviepy - video ready output_video_with_audio_1.5x.mp4
